In [1]:
# import libraries
import pandas as pd
pd.set_option('display.max_row', 100)
pd.set_option('display.max_columns', 100)
import numpy as np
import helper_functions
import ref_files_access
from required_columns import define_required_columns
import data_consolidation
import general_preprocessing
%load_ext autoreload
%autoreload 2

In [64]:
# define refresh parameters
data_dump = r"C:\Users\elaiza.padilla\OneDrive - Accenture\Documents\BSC Intake Optimization\Data"
SOWs = [
    'Commercial Intake Fax'
    ,'Promise Intake Fax'
    ,'Commercial Intake Phone'
    ,'Promise Intake Phone'
    ,'Intake Pharmacy'
    ,'UM RxMed'
    ,'Promise Pharmacy'
]
available_data = {
    # 'Commercial Intake Fax':['Image', 'DPL', 'PA Inventory', 'Commercial Call Back']
    # ,'Promise Intake Fax':['Image', 'DPL', 'PA Inventory', 'Commercial Call Back']
    # ,'Commercial Intake Phone':['DPL']
    # ,'Promise Intake Phone':['DPL']
    # ,'Intake Pharmacy':['Image', 'DPL']
    # ,'UM RxMed':['Image', 'DPL']
    # ,'Promise Pharmacy':['Image', 'DPL']
    'Commercial Intake Fax':[['Image', 'Image Report']]
    ,'Promise Intake Fax':[['Image', 'Image Report']]
    ,'Commercial Intake Phone':[['DPL', 'Intake Phone']]
    ,'Promise Intake Phone':[['DPL', 'Intake Phone']]
    ,'Intake Pharmacy':[['Image', 'Image Report']]
    ,'UM RxMed':[['Image', 'Image Report']]
    ,'Promise Pharmacy':[['Image', 'Image Report']]
}

In [68]:
# data consolidation
master = pd.DataFrame()
for SOW in SOWs:
    # initialize master frame
    master_sow = pd.DataFrame()

    # list available data per SOW
    data_sets = available_data[SOW]

    # read reference files for image data    
    ref_TAT, ref_BD, ref_AHT = ref_files_access.get_ref_files(data_dump)

    for data_set in data_sets:
        report_type = data_set[0]
        sheet_name = data_set[1]

        subfolder_name = helper_functions.get_subfolder_name(SOW, report_type)
        data_files_loc = data_dump + "\\" + report_type + "\\" + subfolder_name + "\\"
        data_files = helper_functions.get_filenames(data_files_loc)

        # consolidate data per data set type
        if report_type == 'Image':
            image = data_consolidation.consolidate_image_data(SOW, data_files_loc, data_files, sheet_name, ref_BD, ref_TAT)
        
        if report_type == 'DPL':
            dpl = data_consolidation.consolidate_dpl_data(SOW, data_files_loc, data_files, sheet_name, skiprows=5, ref_BD=ref_BD, ref_TAT=ref_TAT)

        # if report_type == 'PA Inventory':
        #     pa_inventory = data_consolidation.consolidate_pa_inventory(data_files_loc, data_files, sheet_name)

        # if report_type == 'Commercial Call Back':
        #     commercial_call_back = data_consolidation.consolidate_commercial_call_back(data_files_loc, data_files, sheet_name)
        
    # merge available data
    #sow = pd.concat([image, dpl, pa_inventory, commercial_call_back], axis='columns', ignore_index=True)
    #sow = helper_functions.rename_headers()
    if (SOW == 'Commercial Intake Fax') | (SOW == 'Promise Intake Fax') | (SOW == 'Intake Pharmacy') | (SOW == 'UM RxMed') | (SOW == 'Promise Pharmacy'):
        sow = image
    elif (SOW=="Commercial Intake Phone") | (SOW=="Promise Intake Phone"):
        sow = dpl
    
    master = master.append(sow)
    print(master.shape)

# apply general preprocessing
master = general_preprocessing.preprocess_master_df(master, ref_AHT)

# save output
required_columns = define_required_columns()
master[required_columns].to_excel("Staffing_Optimization.xlsx", index=False)

Commercial Intake Fax
(41609, 84)
(41609, 84)
Promise Intake Fax
(45778, 84)
(87387, 86)
Commercial Intake Phone
(22103, 25)
(109490, 101)
Promise Intake Phone
(4547, 25)
(114037, 101)
Intake Pharmacy
(9002, 21)
(123039, 105)
UM RxMed
(5549, 21)
(128588, 105)
Promise Pharmacy
(708, 21)
(129296, 105)


In [406]:
master[required_columns].shape

(102646, 60)

In [407]:
master['Service Line'].value_counts()

Promise Intake Fax       45778
Commercial Intake Fax    41609
Intake Pharmacy           9002
UM RxMed                  5549
Promise Pharmacy           708
Name: Service Line, dtype: int64